## Tutorial: Columnar ClickHouse

In [1]:
print("Yo")

Yo


In [ ]:
import clickhouse_connect

CLICKHOUSE_HOSTNAME = 'clickHouse'
CLICKHOUSE_USER = 'default'
CLICKHOUSE_PASSWORD = 'test'

client = clickhouse_connect.get_client(
    host=CLICKHOUSE_HOSTNAME, port=8123, username=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD)

print("connected to " + CLICKHOUSE_HOSTNAME + "\n")
client.command(
    'CREATE TABLE IF NOT EXISTS new_table (key UInt32, value String, metric Float64) ENGINE MergeTree ORDER BY key')

print("table new_table created or exists already!\n")
print("-------")
print(client.command('show tables'))

client.command('DROP TABLE new_table')

print("-------")
print(client.command('show tables'))
print("-------")


connected to clickHouse

table new_table created or exists already!

-------
new_table
-------

-------
